# Load libraries

In [1]:
# Use this initial code to work in the notebook as if it were a module, that 
# is, to be able to export classes and functions from other subpackages.

import os
import sys

package_path = os.path.abspath('.').split(os.sep + 'notebooks')[0]
if package_path not in sys.path:
    sys.path.append(package_path)

%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from tuyabot_llm import AbsolutePaths

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# Load environment variables from .env
load_dotenv()

True

# Get information from web pages

In [4]:


# Step 1: Scrape the content from apple.com using WebBaseLoader
urls = [
        "https://www.tuya.com.co/como-pago-mi-tarjeta-o-credicompras", 
        "https://www.tuya.com.co/tarjetas-de-credito", 
        "https://www.tuya.com.co/credicompras", 
        "https://www.tuya.com.co/otras-soluciones-financieras", 
        "https://www.tuya.com.co/nuestra-compania", 
        "https://www.tuya.com.co/activacion-tarjeta"
]

# Create a loader for web content
loader = WebBaseLoader(urls)
documents = loader.load()

In [5]:
documents[0].page_content

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n¿Cómo pago mi Tarjeta o Credicompras? | Tuya\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n      Pasar al contenido principal\n    \n\n\n\n\n\n\n\n\n\nYo tengo\n\n\n\n\n\n\n\n\n\n\nSoluciones de pago\n\nTransacciones\n\nQuiero pagar\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMain navigation\n\n\nInicio\n\n\nDisfruta \n\n\nOfertas\n\n\nApp Tuya\n\n\n\n\nDónde pagas\n\n\nProductos y servicios \n\n\nTarjeta de crédito\n\n\nTarjeta Éxito\n\n\nTarjeta Éxito Mastercard\n\n\nTarjeta Carulla Gold Mastercard \n\n\nTarjeta Carulla Mastercard Black\n\n\nTarjeta Viva Mastercard\n\n\nTarjeta Club del Comerciante Mastercard\n\n\nTarjeta Claro Pay\n\n\n\n\nCuenta Digital TuyaPay\n\n\nCrediCompras\n\n\nSeguros y asistencias\n\n\nOtras soluciones financieras\n\n\n\n\nConócenos \n\n\nNuestra compañía\n\n\nGobierno corporativo\n\n\nSostenibilidad\n\n\nAtención a inversionistas\n\n\n\n\nSoluciones de Pa

In [6]:
print(documents[1].page_content)





























Tarjetas de Crédito Tuya: para comprar lo que más te gusta





















































      Pasar al contenido principal
    









Yo tengo










Soluciones de pago

Transacciones

Quiero pagar














Main navigation


Inicio


Disfruta 


Ofertas


App Tuya




Dónde pagas


Productos y servicios 


Tarjeta de crédito


Tarjeta Éxito


Tarjeta Éxito Mastercard


Tarjeta Carulla Gold Mastercard 


Tarjeta Carulla Mastercard Black


Tarjeta Viva Mastercard


Tarjeta Club del Comerciante Mastercard


Tarjeta Claro Pay




Cuenta Digital TuyaPay


CrediCompras


Seguros y asistencias


Otras soluciones financieras




Conócenos 


Nuestra compañía


Gobierno corporativo


Sostenibilidad


Atención a inversionistas




Soluciones de Pago


Activar Tarjeta


Trabaja con nosotros











Menú




Llámanos













Preguntas Frecuentes


Buscar


¿Dónde consultar?


Llámanos


Visítanos















 ¿Qué quieres bu

In [7]:
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# nltk.download('punkt')
# nltk.download('stopwords')

# for document in documents:
#     tokens = word_tokenize(document.page_content)
#     stop_words = set(stopwords.words('spanish'))
#     tokens = [token for token in tokens if token.lower() not in stop_words]
#     document.page_content = ' '.join(tokens)

In [8]:
documents[0].page_content

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n¿Cómo pago mi Tarjeta o Credicompras? | Tuya\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n      Pasar al contenido principal\n    \n\n\n\n\n\n\n\n\n\nYo tengo\n\n\n\n\n\n\n\n\n\n\nSoluciones de pago\n\nTransacciones\n\nQuiero pagar\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMain navigation\n\n\nInicio\n\n\nDisfruta \n\n\nOfertas\n\n\nApp Tuya\n\n\n\n\nDónde pagas\n\n\nProductos y servicios \n\n\nTarjeta de crédito\n\n\nTarjeta Éxito\n\n\nTarjeta Éxito Mastercard\n\n\nTarjeta Carulla Gold Mastercard \n\n\nTarjeta Carulla Mastercard Black\n\n\nTarjeta Viva Mastercard\n\n\nTarjeta Club del Comerciante Mastercard\n\n\nTarjeta Claro Pay\n\n\n\n\nCuenta Digital TuyaPay\n\n\nCrediCompras\n\n\nSeguros y asistencias\n\n\nOtras soluciones financieras\n\n\n\n\nConócenos \n\n\nNuestra compañía\n\n\nGobierno corporativo\n\n\nSostenibilidad\n\n\nAtención a inversionistas\n\n\n\n\nSoluciones de Pa

In [9]:
for document in documents:
    document.page_content = document.page_content.lower()

In [10]:
print(documents[1])

page_content='



























tarjetas de crédito tuya: para comprar lo que más te gusta





















































      pasar al contenido principal
    









yo tengo










soluciones de pago

transacciones

quiero pagar














main navigation


inicio


disfruta 


ofertas


app tuya




dónde pagas


productos y servicios 


tarjeta de crédito


tarjeta éxito


tarjeta éxito mastercard


tarjeta carulla gold mastercard 


tarjeta carulla mastercard black


tarjeta viva mastercard


tarjeta club del comerciante mastercard


tarjeta claro pay




cuenta digital tuyapay


credicompras


seguros y asistencias


otras soluciones financieras




conócenos 


nuestra compañía


gobierno corporativo


sostenibilidad


atención a inversionistas




soluciones de pago


activar tarjeta


trabaja con nosotros











menú




llámanos













preguntas frecuentes


buscar


¿dónde consultar?


llámanos


visítanos















 ¿

In [11]:
for document in documents:
    document.page_content = document.page_content.strip()
    document.page_content = document.page_content.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    document.page_content = ' '.join(document.page_content.split())

In [12]:
documents[1].page_content

'tarjetas de crédito tuya: para comprar lo que más te gusta pasar al contenido principal yo tengo soluciones de pago transacciones quiero pagar main navigation inicio disfruta ofertas app tuya dónde pagas productos y servicios tarjeta de crédito tarjeta éxito tarjeta éxito mastercard tarjeta carulla gold mastercard tarjeta carulla mastercard black tarjeta viva mastercard tarjeta club del comerciante mastercard tarjeta claro pay cuenta digital tuyapay credicompras seguros y asistencias otras soluciones financieras conócenos nuestra compañía gobierno corporativo sostenibilidad atención a inversionistas soluciones de pago activar tarjeta trabaja con nosotros menú llámanos preguntas frecuentes buscar ¿dónde consultar? llámanos visítanos ¿qué quieres buscar hoy? you must have javascript enabled to use this form. buscar quizá estés buscando: usos y beneficios extracto pérdida o hurto, solicitudes de quejas y cancelaciones realización de pagos avances y cupo solicitud y activación de tarjeta 

In [13]:
print(documents[1])

page_content='tarjetas de crédito tuya: para comprar lo que más te gusta pasar al contenido principal yo tengo soluciones de pago transacciones quiero pagar main navigation inicio disfruta ofertas app tuya dónde pagas productos y servicios tarjeta de crédito tarjeta éxito tarjeta éxito mastercard tarjeta carulla gold mastercard tarjeta carulla mastercard black tarjeta viva mastercard tarjeta club del comerciante mastercard tarjeta claro pay cuenta digital tuyapay credicompras seguros y asistencias otras soluciones financieras conócenos nuestra compañía gobierno corporativo sostenibilidad atención a inversionistas soluciones de pago activar tarjeta trabaja con nosotros menú llámanos preguntas frecuentes buscar ¿dónde consultar? llámanos visítanos ¿qué quieres buscar hoy? you must have javascript enabled to use this form. buscar quizá estés buscando: usos y beneficios extracto pérdida o hurto, solicitudes de quejas y cancelaciones realización de pagos avances y cupo solicitud y activació

# Chunk files and create embeddings

## Chunk files

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=5,
    length_function=len
)
docs = text_splitter.split_documents(documents)

In [15]:
docs

[Document(metadata={'source': 'https://www.tuya.com.co/como-pago-mi-tarjeta-o-credicompras', 'title': '¿Cómo pago mi Tarjeta o Credicompras? | Tuya', 'description': '*/', 'language': 'es'}, page_content='¿cómo pago mi tarjeta o credicompras? | tuya pasar al contenido principal yo tengo soluciones de pago transacciones quiero pagar main navigation inicio disfruta ofertas app tuya dónde pagas productos y servicios tarjeta de crédito tarjeta éxito tarjeta éxito mastercard tarjeta carulla gold mastercard tarjeta carulla mastercard black tarjeta viva mastercard tarjeta club del comerciante mastercard tarjeta claro pay cuenta digital tuyapay credicompras seguros y asistencias otras soluciones financieras conócenos nuestra compañía gobierno corporativo sostenibilidad atención a inversionistas soluciones de pago activar tarjeta trabaja con nosotros menú llámanos preguntas frecuentes buscar ¿dónde'),
 Document(metadata={'source': 'https://www.tuya.com.co/como-pago-mi-tarjeta-o-credicompras', 't

In [16]:
len(docs)

88

## Embeddings

In [17]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\mario\AppData\Local\Temp\ipykernel_11032\859374440.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create vector database

In [18]:
# Define the persistent directory
current_dir = AbsolutePaths().get_abs_path_folder('raw')
db_dir = os.path.join(current_dir, "tuya_collection")
persistent_directory = os.path.join(db_dir, "chroma_tuya_collection")

# Step 4: Create and persist the vector store with the embeddings
if not os.path.exists(persistent_directory):
    print(f"\n--- Creating vector store in {persistent_directory} ---")
    db = Chroma.from_documents(docs, embeddings, persist_directory=persistent_directory)
    print(f"--- Finished creating vector store in {persistent_directory} ---")
else:
    # print(f"Vector store {persistent_directory} already exists. No need to initialize.")
    # db = Chroma(persist_directory=persistent_directory, embedding_function=embeddings)

    print(f"\n--- Creating vector store in {persistent_directory} ---")
    db = Chroma.from_documents(docs, embeddings, persist_directory=persistent_directory)
    print(f"--- Finished creating vector store in {persistent_directory} ---")


--- Creating vector store in c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\data\raw\tuya_collection\chroma_tuya_collection ---
--- Finished creating vector store in c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\data\raw\tuya_collection\chroma_tuya_collection ---


# Retrive documents

In [19]:
# Step 5: Query the vector store
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10},
)

# Define the user's question
query = "¿Cuáles son los valores la tasa de interés y póliza del producto credicompras?"

# Retrieve relevant documents based on the query
relevant_docs = retriever.invoke(query)

# Display the relevant results with metadata
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")


--- Relevant Documents ---
Document 1:
CrediCompras Dirigido







Con este crédito puedes comprar productos en los almacenes Éxito en las categorías de: electrodomésticos, tecnología, colchones, llantas, muebles, máquinas de ejercicio, bicicletas, motos y movilidad eléctrica.



Características



Te ofrecemos montos desde $1.000.000 hasta $15’000,000 y plazos entre 12 y 36 meses.


Se te hará entrega de una tarjeta de único uso. Con ella pagarás tu compra y tendrás 8 días para utilizarla.




Condiciones del producto.


Debes usar el monto aprobado en una única transacción de compra. En ella puedes llevar diferentes productos de las categorías mencionadas.

Source: https://www.tuya.com.co/credicompras

Document 2:
24,89805% póliza colectiva seguro deudores** $3.143 credicompras libre destino tasa efectiva anual 24,89805% póliza colectiva seguro deudores** $3.143 credicompras bajo monto tasa efectiva anual 56,08602% póliza colectiva seguro deudores $3.143 ** para créditos desembolsa

In [20]:
relevant_docs

[Document(metadata={'description': 'Con nuestro crédito Credicompras podrás tener el televisor, nevera y más productos de la línea electrodigital que deseas de nuestros almacenes Éxito. Ingresa aquí y conoce más.', 'language': 'es', 'source': 'https://www.tuya.com.co/credicompras', 'title': 'Credicompras | Tuya'}, page_content='CrediCompras Dirigido\n\n\n\n\n\n\n\nCon este crédito puedes comprar productos en los almacenes Éxito en las categorías de: electrodomésticos, tecnología, colchones, llantas, muebles, máquinas de ejercicio, bicicletas, motos y movilidad eléctrica.\n\n\n\nCaracterísticas\n\n\n\nTe ofrecemos montos desde $1.000.000 hasta $15’000,000 y plazos entre 12 y 36 meses.\n\n\nSe te hará entrega de una tarjeta de único uso. Con ella pagarás tu compra y tendrás 8 días para utilizarla.\n\n\n\n\nCondiciones del producto.\n\n\nDebes usar el monto aprobado en una única transacción de compra. En ella puedes llevar diferentes productos de las categorías mencionadas.'),
 Document(m

In [21]:
def format_docs(docs):
    formatted_docs = "\n\n".join(doc.page_content for doc in docs)
    # print(f"**Formatted Docs**: {formatted_docs}\n*******************************")  # Inspeccionar la salida de format_docs
    return formatted_docs

context = format_docs(relevant_docs)  # Aplica el formateo a los documentos recuperados
print("Context for LLM:", context)  # Ver el contexto final que se pasará al modelo

Context for LLM: CrediCompras Dirigido







Con este crédito puedes comprar productos en los almacenes Éxito en las categorías de: electrodomésticos, tecnología, colchones, llantas, muebles, máquinas de ejercicio, bicicletas, motos y movilidad eléctrica.



Características



Te ofrecemos montos desde $1.000.000 hasta $15’000,000 y plazos entre 12 y 36 meses.


Se te hará entrega de una tarjeta de único uso. Con ella pagarás tu compra y tendrás 8 días para utilizarla.




Condiciones del producto.


Debes usar el monto aprobado en una única transacción de compra. En ella puedes llevar diferentes productos de las categorías mencionadas.

24,89805% póliza colectiva seguro deudores** $3.143 credicompras libre destino tasa efectiva anual 24,89805% póliza colectiva seguro deudores** $3.143 credicompras bajo monto tasa efectiva anual 56,08602% póliza colectiva seguro deudores $3.143 ** para créditos desembolsados a partir del 1 de abril de 2019. póliza colectiva seguro deudores para crédit

# Load LLM

In [22]:
import torch

if torch.cuda.is_available():
    print("CUDA está disponible. El modelo puede ejecutarse en GPU.")
else:
    print("CUDA no está disponible. El modelo se ejecutará en CPU.")

CUDA está disponible. El modelo puede ejecutarse en GPU.


In [23]:
torch.set_default_tensor_type("torch.cuda.FloatTensor")

c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\torch\__init__.py:1236: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\tensor\python_tensor.cpp:436.)
  _C._set_default_tensor_type(t)


In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline 
from langchain.llms import HuggingFacePipeline 
model_id = 'unsloth/Llama-3.2-1B-Instruct' 
tokenizer = AutoTokenizer.from_pretrained(model_id, device="cuda:0")
model = AutoModelForCausalLM.from_pretrained(model_id) 


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=200,
    truncate=True,
    temperature=0.1,
    top_k=10,
    # repetition_penalty=1.5,
    # no_repeat_ngram_size=4,  # Ajusta el tamaño de los n-gramas que no se pueden repetir
)

#conversión a uso api tipo langchain 
local_llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
C:\Users\mario\AppData\Local\Temp\ipykernel_11032\1051470404.py:21: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=pipe)


In [25]:
# Verifica la salida generada
# prompt = "Dime un chiste"
# response = local_llm(prompt)
# print(response)

# Use RAG architecture

In [46]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """
Eres un agente de servicio al cliente que trabaja para TUYA SA, una empresa que se
dedica a ser la solución financiera del retail y que busca apoyar a los sectores
vulnerables de la sociedad. 

Como agente de servicio al cliente, debes suministrar respuestas amigables y
claras a los clientes.

Emplea el contexto que te ofrece la empresa TUYA delimitado por triple comillas invertidas, para responder
la pregunta que se encuentra al final delimitada por comillas simples.

Siempre que puedas responder con una serie de items hazlo, tu respuesta es máximo de 15 palabras.

Contexto: ```{context}```

Pregunta: '{question}'
"""

prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | local_llm
    | StrOutputParser()
)

In [47]:
# question = "Cuáles son los productos financieros de Tuya?"
# rag_chain.invoke(question)

In [48]:
q = """
¿Cuáles son los valores de la tasa de interés y póliza del producto credicompras?
"""
for chunk in rag_chain.stream(q):
    print(chunk, end="", flush=True)

Respuesta:
Los valores de la 

KeyboardInterrupt: 

In [ ]:
q = """
¿Cuáles son los nombres de las tarjetas de crédito que tiene disponible Tuya?
"""
for chunk in rag_chain.stream(q):
    print(chunk, end="", flush=True)

In [ ]:
import os
import gradio as gr
from transformers import pipeline

In [38]:
import gradio as gr
from langchain.schema import AIMessage, HumanMessage, SystemMessage

system_message = "you act like an astronaut"

def stream_response(message, history):
    print(f"Input: {message}. History: {history}\n")

    history_langchain_format = []
    history_langchain_format.append(SystemMessage(content=system_message))

    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    if message is not None:
        history_langchain_format.append(HumanMessage(content=message))
        partial_message = ""
        for response in rag_chain.stream(history_langchain_format):
            partial_message += response.content
            yield partial_message


demo_interface = gr.ChatInterface(stream_response, textbox=gr.Textbox(placeholder="Send to the LLM...",
                       container=False,
                       autoscroll=True,
                       scale=7),
)

demo_interface.launch(debug=True, share=True)

c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\components\chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://e36b16ad20c2589203.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Input: Hola chatbot de tuya. History: []



Traceback (most recent call last):
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\blocks.py", line 2108, in process_api
    result = await self.call_function(
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\blocks.py", line 1667, in call_function
    prediction = await utils.async_iteration(iterator)
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\utils.py", line 735, in async_iteration
    return await anext(iterator)
  File "c:\Users\mari

Input: Hola otra vez. History: []



Traceback (most recent call last):
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\blocks.py", line 2108, in process_api
    result = await self.call_function(
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\blocks.py", line 1667, in call_function
    prediction = await utils.async_iteration(iterator)
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\utils.py", line 735, in async_iteration
    return await anext(iterator)
  File "c:\Users\mari

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://cbdf3c4af978c0d606.gradio.live
Killing tunnel 127.0.0.1:7860 <> https://9c20fc2a84f896202c.gradio.live
Killing tunnel 127.0.0.1:7860 <> https://e36b16ad20c2589203.gradio.live


In [39]:
demo_interface.close()

Closing server running on port: 7860


In [44]:
template = """
    Eres un agente de servicio al cliente que trabaja para TUYA SA, una empresa que se
    dedica a ser la solución financiera del retail y que busca apoyar a los sectores
    vulnerables de la sociedad. 

    Como agente de servicio al cliente, debes suministrar respuestas amigables y
    claras a los clientes.

    Emplea el contexto que te ofrece la empresa TUYA delimitado por triple comillas invertidas, para responder
    la pregunta que se encuentra al final delimitada por comillas simples.

    Siempre que puedas responder con una serie de items hazlo, tu respuesta es máximo de 15 palabras.

    Contexto: ```{context}```

    Pregunta: '{question}'
"""

prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | local_llm
    | StrOutputParser()
)
    

In [45]:
rag_chain.invoke('Cuál es la misión de tuya?')

ValueError: The following `model_kwargs` are not used by the model: ['truncate'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
import gradio as gr

# Definir la función para generar la respuesta
def generate_response(question):

    global retriever

    
    return rag_chain.invoke(question)

In [43]:
generate_response('Cuál es la misión de tuya?')

ValueError: The following `model_kwargs` are not used by the model: ['truncate'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
# Crear la interfaz de Gradio
demo = gr.Interface(
    fn=generate_response,
    inputs=[
        gr.Textbox(label="Pregunta"),
    ],
    outputs=gr.Textbox(label="Respuesta"),
    title="Agente de Servicio al Cliente",
    description="Pregúntame algo sobre TUYA SA",
)

# Desplegar la interfaz
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://9c20fc2a84f896202c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Traceback (most recent call last):
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\gradio\blocks.py", line 2108, in process_api
    result = await self.call_function(
  File "c:\Users\mar

In [36]:
demo.close()

Closing server running on port: 7860


In [ ]:
def launch(question):
    out = rag_chain.invoke(question)
    return out

In [ ]:
iface = gr.Interface(launch,
                     inputs=gr.Image(type='pil'),
                     outputs="text")